निम्न नोटबुकहरू चलाउनको लागि, यदि तपाईंले अझै गर्नुभएको छैन भने, तपाईंले `text-embedding-ada-002` लाई आधार मोडेलको रूपमा प्रयोग गर्ने मोडेल डिप्लोय गर्न आवश्यक छ र .env फाइल भित्र डिप्लोयमेन्ट नामलाई `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` को रूपमा सेट गर्नुपर्छ।


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

अब, हामी Embedding Index लाई Pandas Dataframe मा लोड गर्न गइरहेका छौं। Embedding Index `embedding_index_3m.json` नामक JSON फाइलमा सुरक्षित गरिएको छ। Embedding Index मा प्रत्येक YouTube ट्रान्सक्रिप्टहरूको Embeddings समावेश गरिएको छ जुन २०२३ को अक्टोबरको अन्त्यसम्मका हुन्।


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

अब, हामी `get_videos` नामको एउटा फंक्शन बनाउन गइरहेका छौं जसले Embedding Index मा सोधिएको प्रश्नको लागि खोजी गर्नेछ। यो फंक्शनले सोधिएको प्रश्नसँग सबैभन्दा मिल्दोजुल्दो शीर्ष ५ भिडियोहरू फिर्ता गर्नेछ। फंक्शन यसरी काम गर्छ:

1. सुरुमा, Embedding Index को एउटा प्रतिलिपि बनाइन्छ।
2. त्यसपछि, सोधिएको प्रश्नको Embedding OpenAI Embedding API प्रयोग गरेर गणना गरिन्छ।
3. त्यसपछि Embedding Index मा `similarity` नामको नयाँ स्तम्भ बनाइन्छ। `similarity` स्तम्भमा सोधिएको प्रश्नको Embedding र प्रत्येक भिडियो खण्डको Embedding बीचको cosine similarity हुन्छ।
4. त्यसपछि, Embedding Index लाई `similarity` स्तम्भद्वारा फिल्टर गरिन्छ। Embedding Index लाई केवल ती भिडियोहरू समावेश हुने गरी फिल्टर गरिन्छ जसको cosine similarity ०.७५ वा सोभन्दा बढी छ।
5. अन्त्यमा, Embedding Index लाई `similarity` स्तम्भअनुसार क्रमबद्ध गरिन्छ र शीर्ष ५ भिडियोहरू फिर्ता गरिन्छ।


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

यो फंक्शन धेरै सरल छ, यसले खोज प्रश्नको नतिजा मात्र प्रिन्ट गर्छ।


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. सुरुमा, Embedding Index लाई Pandas Dataframe मा लोड गरिन्छ।
2. त्यसपछि, प्रयोगकर्तालाई सोधिन्छ कि उनीहरूले कुन प्रश्न सोध्न चाहन्छन्।
3. त्यसपछि `get_videos` फंक्शनलाई Embedding Index मा सोधिएको प्रश्न खोज्नका लागि बोलाइन्छ।
4. अन्त्यमा, `display_results` फंक्शनलाई नतिजा प्रयोगकर्तालाई देखाउनका लागि बोलाइन्छ।
5. त्यसपछि प्रयोगकर्तालाई अर्को प्रश्न सोध्नका लागि सोधिन्छ। यो प्रक्रिया प्रयोगकर्ताले `exit` लेख्दासम्म दोहोरिन्छ।

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ne.png)

तपाईंलाई प्रश्न सोध्नका लागि अनुरोध गरिनेछ। कुनै प्रश्न लेख्नुहोस् र enter थिच्नुहोस्। एप्लिकेशनले सोधिएको प्रश्नसँग सम्बन्धित भिडियोहरूको सूची फर्काउनेछ। एप्लिकेशनले प्रश्नको उत्तर भएको भिडियोको स्थानमा जानका लागि लिंक पनि दिनेछ।

यहाँ केही प्रश्नहरू छन् जसलाई तपाईं प्रयास गर्न सक्नुहुन्छ:

- Azure Machine Learning के हो?
- Convolutional neural networks कसरी काम गर्छन्?
- Neural network के हो?
- के म Jupyter Notebooks लाई Azure Machine Learning सँग प्रयोग गर्न सक्छु?
- ONNX के हो?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**अस्वीकरण**:  
यो दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) प्रयोग गरी अनुवाद गरिएको हो। हामी शुद्धताको लागि प्रयास गर्छौं, तर कृपया ध्यान दिनुहोस् कि स्वचालित अनुवादमा त्रुटिहरू वा अशुद्धताहरू हुन सक्छन्। मूल भाषा मा रहेको मूल दस्तावेज़लाई नै आधिकारिक स्रोत मानिनुपर्छ। महत्वपूर्ण जानकारीको लागि, व्यावसायिक मानव अनुवाद सिफारिस गरिन्छ। यस अनुवादको प्रयोगबाट उत्पन्न हुने कुनै पनि गलतफहमी वा गलत व्याख्याका लागि हामी जिम्मेवार हुने छैनौं।
